In [ ]:
import warnings
warnings.filterwarnings('ignore')

## Loading & Writing data functions

In [ ]:
import json

In [ ]:
def load_data(filename):
  with open(filename,'r',encoding = 'utf-8') as f:
    data = json.load(f)
  return data

def write_data(filename,data):
  with open(filename,'w',encoding = 'utf-8') as f:
    json.dump(filename,data,indent = 4)

In [ ]:
data = load_data('ushmm_dn.json')['texts']
data[0]

' My name David Kochalski. I was born in a small town called , and I was born May 5, 1928.  Well, we were very hard working, six children, father and mother and we had a small mill, flour, buckwheat. We were not prosperous but comfortable.  I went to two schools. One was a public school in the morning. In the afternoon I went to a religious school until almost late at night.  Yes.  Well, I raised in the spirit of Judaism.  No, the school itself, in this little city, was segregated between Catholics and Jews. Mind you, it was a small town, and I would say the majority of the people in that small town were Jewish people. Inside the town, somehow, I don\'t know why, but they separated us Jewish children and Catholic children. As you know, most of the people in Poland were Catholic.  Yes, I used to have friends.  Yes, I did. I felt it, maybe not personally, but I knew of a lot of incidents whereby either they were small little -- I would call it -- we were separated, in other words, but ha

## Text Preprocessing

In [ ]:
# nltk
import nltk
from nltk import word_tokenize, pos_tag
nltk.download('stopwords')
from nltk.corpus import stopwords
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer, PorterStemmer
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import wordnet as wn # for pos

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


### Funcitions for text preprocessing

In [ ]:
import re
from collections import defaultdict
from bs4 import BeautifulSoup

In [ ]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]') # removes alphaneumerics
REPLACE_NUMBERS = re.compile('\d')
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
 '''
    This function takes raw text as input and returns texts with stopwords,symbols,pucntuations,html tags removed
 '''
 text = BeautifulSoup(text, "lxml").text # HTML decoding
 text = text.lower() # lowercase text
 text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text
 text = BAD_SYMBOLS_RE.sub('', text) # delete symbols which are in BAD_SYMBOLS_RE from text
 text = REPLACE_NUMBERS.sub('',text) # replace numbers
 text = ' '.join(word for word in text.split() if word not in STOPWORDS) # delete stopwors from text
 return text

def stemming(sentence):
 
 stemmer = PorterStemmer()
 stemmed = [stemmer.stem(word) for word in sentence.split()]
 #stemSentence = stemSentence.strip()
 return ' '.join(stemmed)


# Lemmatizer function
tag_map = defaultdict(lambda : wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV
lmtzr = WordNetLemmatizer()

def lemmatize(sentence):
  lemmatized_sentence = []
  tokens = word_tokenize(sentence)
  for token, tag in pos_tag(tokens):  # returns token and corresponding tag from list of tokens
    lemma = lmtzr.lemmatize(token, tag_map[tag[0]])
    #print(token, "=>", lemma)
    lemmatized_sentence.append(lemma)
  return ' '.join(lemmatized_sentence)

### Preprocessing input texts

In [ ]:
data_cleaned = [clean_text(text) for text in data]
data_cleaned[0] # data after preprocessing

'name david kochalski born small town called born may well hard working six children father mother small mill flour buckwheat prosperous comfortable went two schools one public school morning afternoon went religious school almost late night yes well raised spirit judaism school little city segregated catholics jews mind small town would say majority people small town jewish people inside town somehow dont know separated us jewish children catholic children know people poland catholic yes used friends yes felt maybe personally knew lot incidents whereby either small little would call separated words hardly got together incidents incidents pleasant incidents called house people regardless religious believe really religious people lovely family yes zionists city socialists communists even though time communists go underground religious institutions parents would say zionists religious aware going even though young youngster father used subscribe many newspapers jewish origin nonjewish pa

In [ ]:
data[0] # original data

' My name David Kochalski. I was born in a small town called , and I was born May 5, 1928.  Well, we were very hard working, six children, father and mother and we had a small mill, flour, buckwheat. We were not prosperous but comfortable.  I went to two schools. One was a public school in the morning. In the afternoon I went to a religious school until almost late at night.  Yes.  Well, I raised in the spirit of Judaism.  No, the school itself, in this little city, was segregated between Catholics and Jews. Mind you, it was a small town, and I would say the majority of the people in that small town were Jewish people. Inside the town, somehow, I don\'t know why, but they separated us Jewish children and Catholic children. As you know, most of the people in Poland were Catholic.  Yes, I used to have friends.  Yes, I did. I felt it, maybe not personally, but I knew of a lot of incidents whereby either they were small little -- I would call it -- we were separated, in other words, but ha

In [ ]:
data_cleaned = [lemmatize(texts) for texts in data_cleaned]
data_cleaned[0]

'name david kochalski bear small town call born may well hard work six child father mother small mill flour buckwheat prosperous comfortable go two school one public school morning afternoon go religious school almost late night yes well raise spirit judaism school little city segregate catholic jew mind small town would say majority people small town jewish people inside town somehow dont know separated u jewish child catholic child know people poland catholic yes use friend yes felt maybe personally knew lot incident whereby either small little would call separated word hardly get together incident incident pleasant incident call house people regardless religious believe really religious people lovely family yes zionist city socialists communist even though time communist go underground religious institution parent would say zionist religious aware go even though young youngster father use subscribe many newspaper jewish origin nonjewish paper paper nothing u however father brother r

## Creating the LDA model

In [ ]:
import gensim
import gensim.corpora as corpora

### Further preprocessing with Gensim's simple_preprocess

In [ ]:
def gen_words(texts):
    '''
      Input : lists of cleaned texts
      Output : list of words
    '''
    final = []
    for text in texts:
        new = gensim.utils.simple_preprocess(text, deacc=True)
        final.append(new)
    return (final)

data_words = gen_words(data_cleaned)

print (data_words[0][0:20])

['name', 'david', 'kochalski', 'bear', 'small', 'town', 'call', 'born', 'may', 'well', 'hard', 'work', 'six', 'child', 'father', 'mother', 'small', 'mill', 'flour', 'buckwheat']


In [ ]:
len(data_words[0])

5321

In [ ]:
len(data_words)

252

### Making Bigrams & Trigrams for better identification of important words for topics

Making Bigrams & Trigrams will help in understanding words that are frequently occuring together in significant proportions in the document

In [ ]:
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]]) # note the ngramms are repsented with _ joing 2 or 3 words

['name', 'david', 'kochalski', 'bear', 'small', 'town', 'call', 'born', 'may', 'well', 'hard', 'work', 'six', 'child', 'father', 'mother', 'small', 'mill', 'flour', 'buckwheat', 'prosperous', 'comfortable', 'go', 'two', 'school', 'one', 'public', 'school', 'morning', 'afternoon', 'go', 'religious', 'school', 'almost', 'late', 'night', 'yes', 'well', 'raise', 'spirit', 'judaism', 'school', 'little', 'city', 'segregate', 'catholic', 'jew', 'mind', 'small', 'town', 'would', 'say', 'majority', 'people', 'small', 'town', 'jewish', 'people', 'inside', 'town', 'somehow', 'dont', 'know', 'separated', 'jewish', 'child', 'catholic', 'child', 'know', 'people', 'poland', 'catholic', 'yes', 'use', 'friend', 'yes', 'felt', 'maybe', 'personally', 'knew', 'lot', 'incident', 'whereby', 'either', 'small', 'little', 'would', 'call', 'separated', 'word', 'hardly', 'get', 'together', 'incident', 'incident', 'pleasant', 'incident', 'call', 'house', 'people', 'regardless', 'religious', 'believe', 'really', '

In [79]:
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]


data_words_bigrams = make_bigrams(data_words)
data_words_bigrams_trigrams = make_trigrams(data_words_bigrams)
print(data_words_bigrams_trigrams[0])

['name', 'david', 'kochalski', 'bear', 'small', 'town', 'call', 'born', 'may', 'well', 'hard', 'work', 'six', 'child', 'father', 'mother', 'small', 'mill', 'flour', 'buckwheat', 'prosperous', 'comfortable', 'go', 'two', 'school', 'one', 'public', 'school', 'morning', 'afternoon', 'go', 'religious', 'school', 'almost', 'late', 'night', 'yes', 'well', 'raise', 'spirit', 'judaism', 'school', 'little', 'city', 'segregate', 'catholic', 'jew', 'mind', 'small', 'town', 'would', 'say', 'majority', 'people', 'small', 'town', 'jewish', 'people', 'inside', 'town', 'somehow', 'dont', 'know', 'separated', 'jewish', 'child', 'catholic', 'child', 'know', 'people', 'poland', 'catholic', 'yes', 'use', 'friend', 'yes', 'felt', 'maybe', 'personally', 'knew', 'lot', 'incident', 'whereby', 'either', 'small', 'little', 'would', 'call', 'separated', 'word', 'hardly', 'get', 'together', 'incident', 'incident', 'pleasant', 'incident', 'call', 'house', 'people', 'regardless', 'religious', 'believe', 'really', '

There's still a problem. Our corpus may still have irrelevant words that are insignificant but are repeated a no.of times. These words won't be helping to identify different topics. In order to remove those words we'll use TF-IDF removal

<a href ='https://stackoverflow.com/questions/24688116/how-to-filter-out-words-with-low-tf-idf-in-a-corpus-with-gensim/35951190'>Reference</a>


In [93]:
doc_list = data_words_bigrams_trigrams
dictionary = corpora.Dictionary(doc_list) # id2word
corpus = [dictionary.doc2bow(doc) for doc in doc_list]
tfidf = gensim.models.TfidfModel(corpus, id2word = dictionary)


#Filter low value words and also words missing in tfidf models.

low_value = 0.02

for i in range(0, len(corpus)):
    bow = corpus[i]
    low_value_words = [] #reinitialize to be safe. You can skip this.
    tfidf_ids = [id for id, value in tfidf[bow]]
    bow_ids = [id for id, value in bow]
    low_value_words = [id for id, value in tfidf[bow] if value < low_value]
    words_missing_in_tfidf = [id for id in bow_ids if id not in tfidf_ids] # The words with tf-idf socre 0 will be missing

    new_bow = [b for b in bow if b[0] not in low_value_words and b[0] not in words_missing_in_tfidf]  
    #reassign        
    corpus[i] = new_bow

len(corpus)

252

In [81]:
id2word = corpora.Dictionary(data_words_bigrams_trigrams)

In [100]:

lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=8,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha="auto")

In [ ]:
!pip install pyLDAvis

     |████████████████████████████████| 1.7 MB 5.2 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 15.7 MB 180 kB/s 
     |████████████████████████████████| 11.3 MB 60.2 MB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136897 sha256=5d14cdce3b70d30f862d2f46614641552e36de3ebe5c901617c3f02ebfe300e2
  Stored in directory: /root/.cache/pip/wheels/c9/21/f6/17bcf2667e8a68532ba2fbf6d5c72fdf4c7f7d9abfa4852d2f
Successfully built pyLDAvis
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
  Attempting uninstall: pandas
    Found existing installation: pandas 1.1.5
    Uninstalling pandas-1.1.5:
      Successfully uninstalled pandas-1.1.5
ERROR: pip's dependency resolver does not currently take int

In [101]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()

# feed the LDA model into the pyLDAvis instance
lda_viz = gensimvis.prepare(lda_model, corpus, id2word)
lda_viz

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.210066  0.099148       1        1  23.139338
4     -0.182226  0.085554       2        1  17.281392
1      0.134219  0.093961       3        1  15.346156
5     -0.158972 -0.010424       4        1  13.071151
6     -0.107630 -0.080747       5        1  10.131012
2      0.114532 -0.245230       6        1   8.444472
0      0.008646  0.035404       7        1   7.321183
7     -0.018636  0.022334       8        1   5.265295, topic_info=        Term          Freq         Total Category  logprob  loglift
3947      uh  13482.000000  13482.000000  Default  30.0000  30.0000
390   ghetto   2352.000000   2352.000000  Default  29.0000  29.0000
143     camp   1653.000000   1653.000000  Default  28.0000  28.0000
689       ph   1580.000000   1580.000000  Default  27.0000  27.0000
752   really    869.000000    869.000000  Default  26.0000  26.0000
...      ...           ...           ...      ...      ...      ...
390   ghetto     53.943357   2352.278807   Topic8  -5.6009  -0.8312
579    march     34.865600    360.971131   Topic8  -6.0374   0.6067
1859      mr     30.106874    144.440946   Topic8  -6.1841   1.3759
447     hide     30.843144    400.830851   Topic8  -6.1600   0.3794
1142  arrive     28.923204    185.729288   Topic8  -6.2242   1.0844

[607 rows x 6 columns], token_table=       Topic      Freq       Term
term                             
3562       1  0.090878         aa
3562       2  0.469538         aa
3562       5  0.015146         aa
3562       8  0.439245         aa
7378       1  0.110507        abe
...      ...       ...        ...
1063       3  0.102931        yes
1063       6  0.001716        yes
1063       7  0.308793        yes
39935      6  0.974309  yoethomas
3098       3  0.941270       youd

[1373 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 5, 2, 6, 7, 3, 1, 8])